# Heuristic V1

In [24]:
import pickle
import numpy as np
import StrokeDef
import xmlparse
import xmltodict
import score_strokes

from StrokeDef import loadStrokeDict
from xmlparse import loadGeometryBases
from score_strokes import strokeErrorScaled

"""

"""

'\n\n'

## Comparing Gene Strokes to Archetypal Strokes

In [25]:
archetypal_strokes = loadStrokeDict()
#for key in archetypal_strokes:
    #print(archetypal_strokes[key].arial)

"""

Parameters:
    gene - .gene file
    OR list of strokes, total num of strokes

Return Value:
    total_array - an array containing the total percentages
    
    percent_arrays - a 2D array
"""
#def getGeneStrokePercentages():

    # char_data = loadGeometryBases(data_dir)



"""
Gives a list of scores for how well a given gene stroke matches each of the archetypal strokes

Parameters:
    geneStroke - the gene stroke to be compared to all the archetypal strokes
    
Return Value:
    strokePercentages - an array containing 51 elements, each representing how well geneStroke
    compares to each of the archetypal strokes
"""
def getStrokePercentage(g_points, g_frac_prog):
    
    errors = [] # need to convert errors to scores, what is maximum error?  
    for key in archetypal_strokes:
        points = convertToPointList(archetypal_strokes[key].arial)
        frac_prog = getFractionalProgress(points)
        error = strokeErrorScaled(g_points, points, g_frac_prog, frac_prog)
        errors.append(error)
    return errors    
    

# unpickle, figure out the format of StrokeType object
# get the p_stroke and p_ref necessary for strokeError
# use strokeErrorScaled?

"""
Parameters:
    stroke_arial - arial data from an archetypal stroke to be converted
    
Return Value:
    point_list - an array of pairs, stroke_arial converted to the point list format of a gene stroke
"""
def convertToPointList(stroke_arial):
    point_list = []
    for segment in stroke_arial:
        point_list.append(np.array(segment))
            
    return point_list

"""
"""
def getFractionalProgress(point_list):
    frac_dists = []
    for segment in point_list:
        print(segment)
        dists = [0]
        for i in range(len(segment)-1):
            dists.append(np.linalg.norm((segment[i]-segment[i+1]))+dists[-1])
        dists = np.array(dists)
        print(dists)
        print(dists.max())
        dists /= dists.max()
        frac_dists.append(dists)
    return frac_dists


"""
*Might already be a pre-existing function for this*
Makes it so that the elements of a given list sum to 1,
by dividing each element by the sum of all the elements

Parameters:
    list - list to be manipulated

Return value
    stand_list - list whose elements sum to 1
"""
#def sumToOne()

'\n*Might already be a pre-existing function for this*\nMakes it so that the elements of a given list sum to 1,\nby dividing each element by the sum of all the elements\n\nParameters:\n    list - list to be manipulated\n\nReturn value\n    stand_list - list whose elements sum to 1\n'

## Comparing Gene Stroke Percentage to Archetype Stroke Counts

In [3]:
"""
Compares the stroke percentages of a gene to the stroke counts of every archetype in order to
find the top 100 archetypes that most closely resemble the gene

Parameters:
    geneStrokePercents - an array containing the total percentages
    corresponding to the number of archetypal strokes present in the gene
    
    numStrokes - number of strokes contained in the gene

Return Value:
    archetype_matches - a list of the top 50 archetypes whose stroke counts most closely match the gene's stroke percentages

"""
def strokeCountComparison()

SyntaxError: invalid syntax (<ipython-input-3-a928a5df63e4>, line 15)

## Check Archetypes for Highest Score

In [ ]:
"""
IDEAS:
"""

In [18]:
# Test archetypal stroke data
stroke_name = "HZZZG"
archetypal_strokes = loadStrokeDict()
print(archetypal_strokes[stroke_name].arial)
points = convertToPointList(archetypal_strokes[stroke_name].arial)
print(points)
print()
frac_prog = getFractionalProgress(points)
print(frac_prog)

[[[0.   0.  ]
  [0.65 0.  ]
  [0.65 0.  ]]

 [[0.65 0.  ]
  [0.3  0.5 ]
  [0.3  0.5 ]]

 [[0.3  0.5 ]
  [0.4  0.35]
  [0.5  0.35]]

 [[0.5  0.35]
  [0.75 0.35]
  [0.75 0.35]]

 [[0.75 0.35]
  [0.65 1.  ]
  [0.65 1.  ]]

 [[0.65 1.  ]
  [0.3  0.9 ]
  [0.3  0.9 ]]]
[array([[0.  , 0.  ],
       [0.65, 0.  ],
       [0.65, 0.  ]]), array([[0.65, 0.  ],
       [0.3 , 0.5 ],
       [0.3 , 0.5 ]]), array([[0.3 , 0.5 ],
       [0.4 , 0.35],
       [0.5 , 0.35]]), array([[0.5 , 0.35],
       [0.75, 0.35],
       [0.75, 0.35]]), array([[0.75, 0.35],
       [0.65, 1.  ],
       [0.65, 1.  ]]), array([[0.65, 1.  ],
       [0.3 , 0.9 ],
       [0.3 , 0.9 ]])]

[[0.   0.  ]
 [0.65 0.  ]
 [0.65 0.  ]]
[0.   0.65 0.65]
0.65
[[0.65 0.  ]
 [0.3  0.5 ]
 [0.3  0.5 ]]
[0.         0.61032778 0.61032778]
0.6103277807866851
[[0.3  0.5 ]
 [0.4  0.35]
 [0.5  0.35]]
[0.         0.18027756 0.28027756]
0.28027756377319946
[[0.5  0.35]
 [0.75 0.35]
 [0.75 0.35]]
[0.   0.25 0.25]
0.25
[[0.75 0.35]
 [0.65 1.  ]
 [0.6

In [26]:
# Test getStrokePercentage()

data_dir = "NewGenes"
char_data = loadGeometryBases(data_dir)
g_data, _, base_data, stroke_sets, _, f_names = char_data
for (gl, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
        g, l = gl
        print()
        print(g)
        print()
        for g_points, g_frac in zip(g, l):
            print(g_points)
            print()
            print(g_frac)
            getStrokePercentage(g_points, g_frac)

[array([[ 0.        , 13.89941493],
       [ 3.67516168, 20.42345569],
       [ 5.90425992, 24.38048643],
       [ 9.57942159, 30.90452719]]), array([[11.80851983, 26.94749645],
       [11.80851983, 11.73575608],
       [11.80851983,  6.13966954],
       [11.80851983,  0.54358299]]), array([[16.26671631,  0.        ],
       [24.83588492,  0.        ],
       [27.98830587,  0.        ],
       [31.14072683,  0.        ]]), array([[20.60275406,  1.6390558 ],
       [20.60275406,  7.23514235],
       [20.60275406, 22.44688271],
       [22.8318523 , 26.40391346],
       [22.8318523 , 32.        ]]), array([[13.03590441, 29.43298999],
       [18.2333679 , 29.43298999],
       [23.43083139, 29.43298999],
       [32.        , 29.43298999]]), array([[30.55393656, 18.95191849],
       [25.35647307, 18.95191849],
       [23.12737483, 14.99488774],
       [14.55820622, 14.99488774]])]
frac_dists
[array([0.        , 0.38365173, 0.61634827, 1.        ]), array([0.        , 0.57611688, 0.78805844, 

ValueError: operands could not be broadcast together with shapes (4,2) (2,2) 